In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import numpy as np
import scipy.io
import joblib
import matplotlib.pyplot as plt


In [ ]:
file_path = './datasets/Karabuk_Pavia_Data.mat'
mat = scipy.io.loadmat(file_path)
mat


In [ ]:

Data = mat['PaviaU']
Data


In [ ]:
Data.shape

In [ ]:
X = Data[:, 0:103]


In [ ]:
y = Data[:,103:104]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=Data[:, 103:104], test_size=0.30, random_state=42)


In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_train, y_train

In [ ]:
xg = XGBClassifier()

params = {
          'booster': ['gbtree', 'gblinear', 'dart'],
          'max_depth': [10, 15, 25, 50],
          'eta': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5],
          'n_estimators': [100, 250, 500, 750],
          }

clf = RandomizedSearchCV(xg, param_distributions=params,
                         scoring='accuracy',
                         n_iter=10,
                         cv=3,
                         verbose=1)


clf = clf.fit(X_train, y_train)


In [ ]:
print("Best: %f using %s" % (clf.best_score_, 
    clf.best_params_))
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
y_pred = clf.predict(X_test)

acc = str(format(accuracy_score(y_test, y_pred)*100, '.2f'))

print(acc+"%")

In [ ]:


print(classification_report(y_pred, y_test))

In [ ]:


mat = confusion_matrix(y_test, y_pred)
mat

In [ ]:

joblib.dump(clf, f'./models/XGB_model_ACC_{acc}')

In [ ]:
file_path = './datasets/PaviaU.mat'
mat = scipy.io.loadmat(file_path)
mat


In [ ]:
data = mat['paviaU']

In [ ]:
data.shape

In [ ]:
data[0][0]

In [ ]:
new_data = data.reshape(610*340, 103)

In [ ]:
loaded_clf = joblib.load(f'./models/XGB_model_ACC_{acc}')
loaded_clf

In [ ]:
predictions = loaded_clf.predict(new_data)


In [ ]:
predictions.shape


In [ ]:
image = np.array([np.append(n, np.zeros((1))) for n in new_data])

In [ ]:
for i in range(len(image)):
    image[i][-1] = predictions[i]


In [ ]:
image = image.reshape(610, 340, 104)


In [ ]:
fig = plt.figure(figsize=(10, 8))
plt.imshow(image[:,:,103], cmap='nipy_spectral')
plt.colorbar()
plt.axis('off')
plt.show()
fig.savefig('./images/classified_image.png')